***!!! Warning, ugly code !!!***

This Notebook contains Ideas for the query engine, hence have a lot of unused code. But it is a code base interesting to save.
it contains : 

-> Idea for a custom Retriever
-> Idea for a custom indexation of sub elements
-> Usage of Recursive retriever (not in the final choice)

Learnings : MarkDownElementNodeParser is cutting the retriever capabilities in this use case because of fewer semantic similarity.

In [43]:
from pathlib import Path
import sys
sys.path.append('..')
from megaparse.Converter import MegaParse
import os 

api_key: str | None = os.getenv("LLAMA_CLOUD_API_KEY")

# converter = MegaParse(file_path="../megaparse/cdp/CDP_QUAL_CHART_01_CHARTE PRODUITS_2023.12.13.pdf", llama_parse_api_key=api_key)
# md_content = converter.convert()
# converter.save_md(md_content, Path("../megaparse/cdp/CDP_QUAL_CHART_01_CHARTE_PRODUITS.md"))

In [44]:
import pandas as pd

pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)
pd.set_option("display.max_colwidth", None)

In [60]:
import sys
sys.path.append('../')
from llama_index.readers.file import MarkdownReader
from megaparse.markdown_element import MarkdownElementNodeParser
from llama_index.core.node_parser import MarkdownNodeParser

from pathlib import Path

reader = MarkdownReader()
md_doc = reader.load_data(Path("../megaparse/cdp/CDP_QUAL_CHART_01_CHARTE_PRODUITS.md"))

In [61]:
import asyncio
import nest_asyncio
import uvloop

if not isinstance(asyncio.get_event_loop(), uvloop.Loop):
    nest_asyncio.apply()

In [62]:
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import VectorStoreIndex
from llama_index.core import Settings

embed_model = OpenAIEmbedding(model="text-embedding-3-small")
llm = OpenAI(model="gpt-4o")

Settings.llm = llm
Settings.embed_model = embed_model

#node_parser = MarkdownElementNodeParser(llm=OpenAI(model="gpt-4o"))
node_parser = MarkdownNodeParser()

In [63]:
raw_nodes_charte = node_parser.get_nodes_from_documents(md_doc, show_progress= False)


In [64]:
raw_nodes_charte

[TextNode(id_='fe5d7d26-a564-446a-888b-282e5c49fe3c', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='375d5c41-3c65-40f1-b78f-9df88625c9ca', node_type=<ObjectType.DOCUMENT: '4'>, metadata={}, hash='44136fa355b3678a1146ad16f7e8649e94fb4fc21fe77e8310c060f61caaff8a')}, text='', start_char_idx=0, end_char_idx=0, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'),
 TextNode(id_='8d27f7af-848c-4b9d-9b08-ade1ccd8e383', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='3af156f7-d42d-4613-a4bc-722bc7e602bb', node_type=<ObjectType.DOCUMENT: '4'>, metadata={}, hash='09953039bdb6095b9f45b6bd1cdf14af1c71fcfb86c7482ebc7dad43c35838fd')}, text="CHARTE PRODUITS COUP DE PATES\n\n**Codification**: CDP_QUA_CHART_01  \n**Ver

In [65]:
import os
import pickle
from typing import List

from llama_index.core.schema import BaseNode

if not os.path.exists("../megaparse/cdp/charte.pkl"):
    raw_nodes_charte: List[BaseNode] = node_parser.get_nodes_from_documents(md_doc, show_progress= False)
    pickle.dump(raw_nodes_charte, open("../megaparse/cdp/charte.pkl", "wb"))
else:
    raw_nodes_charte = pickle.load(open("../megaparse/cdp/charte.pkl", "rb"))

In [53]:
base_nodes_charte, node_mappings_charte = node_parser.get_base_nodes_and_mappings(
    raw_nodes_charte
)

AttributeError: 'MarkdownNodeParser' object has no attribute 'get_base_nodes_and_mappings'

**Link nodes to CF.**

In [7]:
from llama_index.core.retrievers import RecursiveRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core import VectorStoreIndex

vector_index = VectorStoreIndex(base_nodes_charte)
vector_retriever = vector_index.as_retriever(similarity_top_k=1)

recursive_retriever = RecursiveRetriever(
    "vector",
    retriever_dict={"vector": vector_retriever},
    node_dict=node_mappings_charte,
    verbose=True,
)
query_engine = RetrieverQueryEngine.from_args(recursive_retriever)


In [25]:
import json
from typing import Any
from llama_index.core.schema import  TextNode, NodeRelationship, RelatedNodeInfo
from typing import Dict, Optional, Union, Tuple
from llama_index.core.bridge.pydantic import BaseModel, Field
from llama_index.core.schema import ObjectType


class RefIndexNode(TextNode):
    """Node with reference to multiple objects.

    This can include other indices, query engines, retrievers.

    This can also include other nodes (though this is overlapping with `relationships`
    on the Node class).

    """

    index_id: str
    obj: List[Any] = None

    def dict(self, **kwargs: Any) -> Dict[str, Any]:
        from llama_index.core.storage.docstore.utils import doc_to_json

        data = super().dict(**kwargs)

        try:
            if self.obj is None:
                data["obj"] = None
            else:
                serialized_objs = []
                for obj in self.obj:
                    if isinstance(obj, BaseNode):
                        serialized_objs.append(doc_to_json(obj))
                    elif isinstance(obj, BaseModel):
                        serialized_objs.append(obj.dict())
                    else:
                        serialized_objs.append(json.dumps(obj))
                data["obj"] = serialized_objs
        except Exception:
            raise ValueError("IndexNode obj is not serializable: " + str(self.obj))

        return data

    @classmethod
    def from_text_node(
        cls,
        node: TextNode,
        index_id: str | None = None,
    ) -> "RefIndexNode":
        """Create index node from text node."""
        # copy all attributes from text node, add index id
        return cls(
            **node.dict(),
            #index_id=index_id,
        )

    # TODO: return type here not supported by current mypy version
    @classmethod
    def from_dict(cls, data: Dict[str, Any], **kwargs: Any) -> "IndexNode":
        from llama_index.core.storage.docstore.utils import json_to_doc

        output = super().from_dict(data, **kwargs)

        obj = data.get("obj", None)
        parsed_obj = []

        if isinstance(obj, list):
            for item in obj:
                if isinstance(item, str):
                    parsed_obj.append(TextNode(text=item))
                elif isinstance(item, dict):
                    try:
                        parsed_obj.append(json_to_doc(item))
                    except Exception:
                        parsed_obj.append(TextNode(text=str(item)))
                else:
                    parsed_obj.append(TextNode(text=str(item)))
        elif obj is not None:
            raise ValueError("Expected obj to be a list")

        output.obj = parsed_obj

        return output

    @classmethod
    def get_type(cls) -> str:
        return ObjectType.INDEX

    @classmethod
    def class_name(cls) -> str:
        return "RefIndexNode"

In [26]:
# from llama_index.core.schema import IndexNode, TextNode, NodeRelationship, RelatedNodeInfo

# import re

# new_raw_nodes_charte = []

# for node in raw_nodes_charte:
#     if isinstance(node, TextNode):
#         content = node.get_content()
#     elif isinstance(node, IndexNode):
#         content = node.relationships['3'].metadata["table_df"] # type: ignore
#     else:
#         raise ValueError("Unknown node type")
    
#     #check if content contains a cf. and get the following text
#     refs = re.findall(r'\(cf\. ([^()]+)\)' , content)
#     if refs:
#         print("okay")
#         before_node = node
        
#         ref_nodes = []
#         for ref in refs :
#             #get the node of the reference 
#             ref_node = query_engine.retrieve(f"What is the content of {ref} ?")[0] #type: ignore
#             ref_nodes.append(ref_node.node)

#         if isinstance(node, TextNode):
#             new_node = RefIndexNode.from_text_node(node, "charte")
#             #add the reference to the node
#             new_node.obj = ref_nodes
            
#             new_raw_nodes_charte.append(new_node)
#         else:
#             new_raw_nodes_charte.append(node)

#     else:
#         new_raw_nodes_charte.append(node)
            


okay
Retrieving with query id None: What is the content of Annexe IV ?
Retrieving text node: ANNEXE IV : Ingrédients controversés : ingrédients faisant l’objet de rapports scientifiques controversés et/ou perçus négativement par le consommateur.
Retrieving with query id None: What is the content of Annexe I ?
Retrieving text node: ANNEXE 1 : Additifs rouges : additifs pour lesquels les rapports scientifiques rapportent une potentielle cancérogénicité ou une implication dans les pathologies lourdes
Retrieving with query id None: What is the content of Annexe II ?
Retrieving text node: ANNEXE II : Additifs oranges : additifs pour lesquels les rapports scientifiques sont contradictoires


In [29]:
new_raw_nodes_charte[10].obj

[TextNode(id_='9ec5005a-62c9-4333-b061-488dcc3870ec', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='ef54882d-e87e-4b21-9979-a4b21a9822d9', node_type=<ObjectType.DOCUMENT: '4'>, metadata={}, hash='c14b16dbf1d39d40a2dfda2292f356f68ec8d6228d384b482bfa8ae0b9f7ce88'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='38c5a0c9-2547-4821-83f4-4f3553b87d0a', node_type=<ObjectType.INDEX: '3'>, metadata={'col_schema': 'Column: Ingrédient\nType: string\nSummary: None\n\nColumn: Motif\nType: string\nSummary: None'}, hash='bc8a2e61c020ec34f743cc44242bba43d7c6aa0ec136b9364bdc5ae3c56fb9ef')}, text='ANNEXE IV : Ingrédients controversés : ingrédients faisant l’objet de rapports scientifiques controversés et/ou perçus négativement par le consommateur.', start_char_idx=2, end_char_idx=154, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_sepe

In [23]:
from llama_index.core.schema import IndexNode, TextNode, NodeRelationship, RelatedNodeInfo

import re

new_base_nodes_charte = []

for node in base_nodes_charte:
    if isinstance(node, IndexNode):
        index_id = node.index_id
        indexed_node = node_mappings_charte[index_id]
        content = indexed_node.metadata['table_df']
    elif isinstance(node, TextNode):
        content = node.get_content()
    
    else:
        raise ValueError("Unknown node type")
    
    #check if content contains a cf. and get the following text
    refs = re.findall(r'\(cf\. ([^()]+)\)' , content)
    if refs:
        print("okay")
        before_node = node
        
        ref_nodes = []
        for ref in refs :
            #get the node of the reference 
            ref_node = query_engine.retrieve(f"What is the content of {ref} ?")[0] #type: ignore
            ref_nodes.append(ref_node.node)

        new_node = RefIndexNode.from_text_node(node, "charte")
        #add the reference to the node
        new_node.obj = ref_nodes
            
        new_base_nodes_charte.append(new_node)

    else:
        new_base_nodes_charte.append(node)

okay
Retrieving with query id None: What is the content of Annexe IV ?
Retrieving text node: ANNEXE IV : Ingrédients controversés : ingrédients faisant l’objet de rapports scientifiques controversés et/ou perçus négativement par le consommateur.
Retrieving with query id None: What is the content of Annexe I ?
Retrieving text node: ANNEXE 1 : Additifs rouges : additifs pour lesquels les rapports scientifiques rapportent une potentielle cancérogénicité ou une implication dans les pathologies lourdes
Retrieving with query id None: What is the content of Annexe II ?
Retrieving text node: ANNEXE II : Additifs oranges : additifs pour lesquels les rapports scientifiques sont contradictoires


In [22]:
new_base_nodes_charte[7].obj

[TextNode(id_='9ec5005a-62c9-4333-b061-488dcc3870ec', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='ef54882d-e87e-4b21-9979-a4b21a9822d9', node_type=<ObjectType.DOCUMENT: '4'>, metadata={}, hash='c14b16dbf1d39d40a2dfda2292f356f68ec8d6228d384b482bfa8ae0b9f7ce88'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='38c5a0c9-2547-4821-83f4-4f3553b87d0a', node_type=<ObjectType.INDEX: '3'>, metadata={'col_schema': 'Column: Ingrédient\nType: string\nSummary: None\n\nColumn: Motif\nType: string\nSummary: None'}, hash='bc8a2e61c020ec34f743cc44242bba43d7c6aa0ec136b9364bdc5ae3c56fb9ef')}, text='ANNEXE IV : Ingrédients controversés : ingrédients faisant l’objet de rapports scientifiques controversés et/ou perçus négativement par le consommateur.', start_char_idx=2, end_char_idx=154, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_sepe

In [30]:
base_nodes_charte, node_mappings_charte = node_parser.get_base_nodes_and_mappings(
    new_raw_nodes_charte
)

In [196]:
from typing import Dict, Optional, Union, Tuple
from llama_index.core.base.base_retriever import BaseRetriever
from llama_index.core.base.base_query_engine import BaseQueryEngine
from llama_index.core.callbacks.base import CallbackManager
from llama_index.core.utils import print_text
from llama_index.core.callbacks.schema import CBEventType, EventPayload
from llama_index.core.schema import (
    BaseNode,
    IndexNode,
    NodeWithScore,
    QueryBundle,
    TextNode,
)

DEFAULT_QUERY_RESPONSE_TMPL = "Query: {query_str}\nResponse: {response}"

RQN_TYPE = Union[BaseRetriever, BaseQueryEngine, BaseNode]

class RecursiveRefRetriever(BaseRetriever):
    """Recursive retriever.

    This retriever will recursively explore links from nodes to other
    retrievers/query engines.

    For any retrieved nodes, if any of the nodes are IndexNodes,
    then it will explore the linked retriever/query engine, and query that.

    Args:
        root_id (str): The root id of the query graph.
        retriever_dict (Optional[Dict[str, BaseRetriever]]): A dictionary
            of id to retrievers.
        query_engine_dict (Optional[Dict[str, BaseQueryEngine]]): A dictionary of
            id to query engines.

    """

    def __init__(
        self,
        root_id: str,
        retriever_dict: Dict[str, BaseRetriever],
        query_engine_dict: Optional[Dict[str, BaseQueryEngine]] = None,
        node_dict: Optional[Dict[str, BaseNode]] = None,
        callback_manager: Optional[CallbackManager] = None,
        query_response_tmpl: Optional[str] = None,
        verbose: bool = False,
    ) -> None:
        """Init params."""
        self._root_id = root_id
        if root_id not in retriever_dict:
            raise ValueError(
                f"Root id {root_id} not in retriever_dict, it must be a retriever."
            )
        self._retriever_dict = retriever_dict
        self._query_engine_dict = query_engine_dict or {}
        self._node_dict = node_dict or {}

        # make sure keys don't overlap
        if set(self._retriever_dict.keys()) & set(self._query_engine_dict.keys()):
            raise ValueError("Retriever and query engine ids must not overlap.")

        self._query_response_tmpl = query_response_tmpl or DEFAULT_QUERY_RESPONSE_TMPL
        super().__init__(callback_manager, verbose=verbose)

    def _deduplicate_nodes(
        self, nodes_with_score: List[NodeWithScore]
    ) -> List[NodeWithScore]:
        """Deduplicate nodes according to node id.
        Keep the node with the highest score/first returned.
        """
        node_ids = set()
        deduplicate_nodes = []
        for node_with_score in nodes_with_score:
            node = node_with_score.node
            if node.id_ not in node_ids:
                node_ids.add(node.id_)
                deduplicate_nodes.append(node_with_score)
        return deduplicate_nodes

    def _query_retrieved_nodes(
        self, query_bundle: QueryBundle, nodes_with_score: List[NodeWithScore]
    ) -> Tuple[List[NodeWithScore], List[NodeWithScore]]:
        """Query for retrieved nodes.

        If node is an IndexNode, then recursively query the retriever/query engine.
        If node is a TextNode, then simply return the node.

        """
        nodes_to_add = []
        additional_nodes = []
        visited_ids = set()

        # dedup index nodes that reference same index id
        new_nodes_with_score = []
        for node_with_score in nodes_with_score:
            node = node_with_score.node
            if isinstance(node, IndexNode):
                if node.index_id not in visited_ids:
                    visited_ids.add(node.index_id)
                    new_nodes_with_score.append(node_with_score)
            else:
                new_nodes_with_score.append(node_with_score)

        nodes_with_score = new_nodes_with_score

        # recursively retrieve
        for node_with_score in nodes_with_score:
            node = node_with_score.node
            if isinstance(node, IndexNode):
                if self._verbose:
                    print_text(
                        "Retrieved node with id, entering: " f"{node.index_id}\n",
                        color="pink",
                    )
                cur_retrieved_nodes, cur_additional_nodes = self._retrieve_rec(
                    query_bundle,
                    query_id=node.index_id,
                    cur_similarity=node_with_score.score,
                )
            elif isinstance(node, RefIndexNode):
                ref_ids = [o.id_ for o in node.obj]
                if self._verbose:
                    print_text(
                        "Retrieved node with refs, entering: " f"{" ".join(ref_ids)}\n",
                        color="pink",
                    )
                for ref_id in ref_ids:
                    cur_retrieved_nodes, cur_additional_nodes = self._retrieve_rec(
                    query_bundle,
                    query_id=ref_id,
                    cur_similarity=node_with_score.score,
                    )

            else:
                assert isinstance(node, TextNode)
                if self._verbose:
                    # print_text(
                    #     "Retrieving text node: " f"{node.get_content()}\n",
                    #     color="pink",
                    # )
                    print_text(
                        "Retrieving text node: " f"{node.get_content()[:20]}\n",
                        color="pink",
                    )
                cur_retrieved_nodes = [node_with_score]
                cur_additional_nodes = []

            nodes_to_add.extend(cur_retrieved_nodes)
            additional_nodes.extend(cur_additional_nodes)

        # dedup nodes in case some nodes could be retrieved from multiple sources
        nodes_to_add = self._deduplicate_nodes(nodes_to_add)
        additional_nodes = self._deduplicate_nodes(additional_nodes)
        return nodes_to_add, additional_nodes

    def _get_object(self, query_id: str) -> RQN_TYPE:
        """Fetch retriever or query engine."""
        node = self._node_dict.get(query_id, None)
        if node is not None:
            return node
        retriever = self._retriever_dict.get(query_id, None)
        if retriever is not None:
            return retriever
        query_engine = self._query_engine_dict.get(query_id, None)
        if query_engine is not None:
            return query_engine
        raise ValueError(
            f"Query id {query_id} not found in either `retriever_dict` "
            "or `query_engine_dict`."
        )

    def _retrieve_rec(
        self,
        query_bundle: QueryBundle,
        query_id: Optional[str] = None,
        cur_similarity: Optional[float] = None,
    ) -> Tuple[List[NodeWithScore], List[NodeWithScore]]:
        """Query recursively."""
        if self._verbose:
            print_text(
                f"Retrieving with query id {query_id}: {query_bundle.query_str}\n",
                color="blue",
            )
        query_id = query_id or self._root_id
        cur_similarity = cur_similarity or 1.0

        obj = self._get_object(query_id)
        if isinstance(obj, BaseNode):
            nodes_to_add = [NodeWithScore(node=obj, score=cur_similarity)]
            additional_nodes: List[NodeWithScore] = []
        elif isinstance(obj, BaseRetriever):
            with self.callback_manager.event(
                CBEventType.RETRIEVE,
                payload={EventPayload.QUERY_STR: query_bundle.query_str},
            ) as event:
                nodes = obj.retrieve(query_bundle)
                event.on_end(payload={EventPayload.NODES: nodes})

            nodes_to_add, additional_nodes = self._query_retrieved_nodes(
                query_bundle, nodes
            )

        elif isinstance(obj, BaseQueryEngine):
            sub_resp = obj.query(query_bundle)
            if self._verbose:
                print_text(
                    f"Got response: {sub_resp!s}\n",
                    color="green",
                )
            # format with both the query and the response
            node_text = self._query_response_tmpl.format(
                query_str=query_bundle.query_str, response=str(sub_resp)
            )
            node = TextNode(text=node_text)
            nodes_to_add = [NodeWithScore(node=node, score=cur_similarity)]
            additional_nodes = sub_resp.source_nodes
        else:
            raise ValueError("Must be a retriever or query engine.")

        return nodes_to_add, additional_nodes

    def _retrieve(self, query_bundle: QueryBundle) -> List[NodeWithScore]:
        retrieved_nodes, _ = self._retrieve_rec(query_bundle, query_id=None)
        return retrieved_nodes

    def retrieve_all(
        self, query_bundle: QueryBundle
    ) -> Tuple[List[NodeWithScore], List[NodeWithScore]]:
        """Retrieve all nodes.
        Unlike default `retrieve` method, this also fetches additional sources.

        """
        return self._retrieve_rec(query_bundle, query_id=None)

In [8]:
from llama_index.core.schema import IndexNode, TextNode, MetadataMode
example_index_node = [b for b in base_nodes_charte if isinstance(b, TextNode)][
    22
]

print(
    f"\n--------\n{example_index_node.get_content(metadata_mode=MetadataMode.ALL)}\n--------\n"
)


--------
col_schema: Column: Additif
Type: string
Summary: None

Column: Code
Type: string
Summary: None

Column: Additif
Type: string
Summary: None

Column: Code
Type: string
Summary: None

This table lists various food additives along with their respective codes, highlighting those that have been reported to have potential carcinogenicity or involvement in serious diseases.,
with the following table title:
ANNEXE I : Additifs rouges,
with the following columns:
- Additif: None
- Code: None
- Additif: None
- Code: None
--------



In [9]:
index_nodes: list[IndexNode] = [b for b in base_nodes_charte if isinstance(b, IndexNode)]


In [17]:
index_nodes[4].get_content()


'**ANNEXE II : Additifs oranges : additifs pour lesquels les rapports scientifiques sont contradictoires**\n\nThis table lists food additives with their corresponding codes, highlighting those for which scientific reports are contradictory. \n\n**Table ID:** Not provided\n\n**Should the table be kept?** Yes,\nwith the following columns:\n'

In [54]:
import pandas as pd
from llama_index.core.schema import IndexNode
import ast

index_nodes: list[IndexNode] = [b for b in base_nodes_charte if isinstance(b, IndexNode)]
data = index_nodes[4].relationships['3'].metadata["table_df"].replace("\'", "'")
data_dict = ast.literal_eval(data)
df = pd.DataFrame(data_dict)
df

,Additif,Code
0,"Acide carminique, carmins",E 120
1,Bleu patenté V,E 131
2,Caramel de sulfite caustique,E 150b
3,Or,E 175
4,Acide benzoïque,E 210
5,Benzoate de sodium,E 211
6,Anhydride sulfureux,E 220
7,Sulfite de sodium,E 221
8,Sulfite acide de sodium,E 222
9,Disulfite de sodium,E 223


In [ ]:
from llama_index.core.schema import IndexNode, TextNode, MetadataMode

example_index_node = [b for b in base_nodes_charte if isinstance(b, IndexNode)][
    2
]
# Index Node
print(
    f"\n--------\n{example_index_node.get_content(metadata_mode=MetadataMode.ALL)}\n--------\n"
)
# Index Node ID
print(f"\n--------\nIndex ID: {example_index_node.index_id}\n--------\n")
# Referenceed Table
print(
    f"\n--------\n{node_mappings_charte[example_index_node.index_id].get_content()}\n--------\n"
)

### Create Ref Nodes

In [66]:
from llama_index.core.schema import IndexNode, TextNode, MetadataMode
cleaned_nodes_charte = [node for node in raw_nodes_charte if node.get_content(metadata_mode=MetadataMode.EMBED) != "" ]

In [67]:
table_nodes = [node for node in cleaned_nodes_charte if "|" in node.get_content()]

In [73]:
table_nodes[4].get_content()

'ANNEXE II : Additifs oranges : additifs pour lesquels les rapports scientifiques sont contradictoires\n\n| Additif                          | Code  | Additif                                                                 | Code  |\n|----------------------------------|-------|------------------------------------------------------------------------|-------|\n| Acide carminique, carmins        | E 120 | Esters lactiques des mono- et diglycérides d’acides gras                | E 472b|\n| Bleu patenté V                   | E 131 | Esters citriques des mono- et diglycérides d’acides gras                | E 472c|\n| Caramel de sulfite caustique     | E 150b| Esters tartriques des mono- et diglycérides d’acides gras               | E 472d|\n| Or                               | E 175 | Esters monoacétyltartriques et diacétyltartriques des mono- et diglycérides d’acides gras | E 472e|\n| Acide benzoïque                  | E 210 | Esters mixtes acétiques et tartriques des mono- et diglycérides 

In [74]:
import re

ref_dict = {}

for node in cleaned_nodes_charte:
    content = node.get_content()
    refs = re.findall(r'\(cf\. ([^()]+)\)', content)
    
    for ref in refs:
        ref_lower = ref.lower() + " "
        
        for table_node in table_nodes:
            if table_node != node and ref_lower in table_node.get_content().lower():
                ref_dict.setdefault(table_node.id_, []).append(node)

In [75]:
ref_dict #ok

{'bd0f1181-6024-4aab-af7a-26cff7008782': [TextNode(id_='b48ed788-64c1-4156-b476-2ab05c19b296', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='012e8592-853a-4a3c-99a5-dc562888f0e7', node_type=<ObjectType.DOCUMENT: '4'>, metadata={}, hash='78f3f962d977fc5955368245fea9524b4257887505299c9f064866cd06c532ff')}, text='1. Exigence recette\n\nPour le développement de nos produits, nous souhaitons favoriser une offre saine avec des recettes simples (sans colorant, sans arôme, sans conservateur), avec des ingrédients de qualité, en favorisant des produits locaux et labellisés.\n\nLe fournisseur s’engage à respecter la réglementation européenne et nationale ainsi que les codes d’usages professionnels applicables aux produits surgelés vendus à Coup de Pates.\n\nPour les produits commercialisés sous une marque appartenant à Coup de Pates, le fournisseur s’engage également à respecter

In [76]:
index_raw_nodes_chart = []

for node in cleaned_nodes_charte:
    if node.id_ in ref_dict:
        if isinstance(node, TextNode):
            new_node = IndexNode.from_text_node(node, ref_dict[node.id_][0].id_)
            index_raw_nodes_chart.append(new_node)
    else:
        index_raw_nodes_chart.append(node)

In [77]:
index_raw_nodes_chart 

[TextNode(id_='0899879b-94ae-4b6f-af2d-6338c77c4513', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='3af156f7-d42d-4613-a4bc-722bc7e602bb', node_type=<ObjectType.DOCUMENT: '4'>, metadata={}, hash='09953039bdb6095b9f45b6bd1cdf14af1c71fcfb86c7482ebc7dad43c35838fd')}, text="CHARTE PRODUITS COUP DE PATES\n\n**Codification**: CDP_QUA_CHART_01  \n**Version**: 5  \n**Date d'application**: 13/12/2023\n\n| Date       | Mises à jour                                                                 |\n|------------|------------------------------------------------------------------------------|\n| 19/12/2014 | Création                                                                     |\n| 12/12/2019 | Insertion des additifs interdits et à éviter                                 |\n| 13/05/2022 | Revue des exigences recettes et annexes                                      |\n| 30/03/

In [78]:
node_parser = MarkdownElementNodeParser(llm=OpenAI(model="gpt-4o"))
base_nodes_charte, node_mappings_charte = node_parser.get_base_nodes_and_mappings(
    index_raw_nodes_chart 
)

In [79]:
base_nodes_charte

[TextNode(id_='0899879b-94ae-4b6f-af2d-6338c77c4513', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='3af156f7-d42d-4613-a4bc-722bc7e602bb', node_type=<ObjectType.DOCUMENT: '4'>, metadata={}, hash='09953039bdb6095b9f45b6bd1cdf14af1c71fcfb86c7482ebc7dad43c35838fd')}, text="CHARTE PRODUITS COUP DE PATES\n\n**Codification**: CDP_QUA_CHART_01  \n**Version**: 5  \n**Date d'application**: 13/12/2023\n\n| Date       | Mises à jour                                                                 |\n|------------|------------------------------------------------------------------------------|\n| 19/12/2014 | Création                                                                     |\n| 12/12/2019 | Insertion des additifs interdits et à éviter                                 |\n| 13/05/2022 | Revue des exigences recettes et annexes                                      |\n| 30/03/

#### Define custom retriever

### Retriever

In [80]:
from llama_index.core.retrievers import RecursiveRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core import VectorStoreIndex

# construct top-level vector index + query engine
vector_index = VectorStoreIndex(cleaned_nodes_charte)
vector_retriever = vector_index.as_retriever(similarity_top_k=5)

In [81]:
# from llama_index.core.retrievers import RecursiveRetriever

# recursive_retriever = RecursiveRetriever(
#     "vector",
#     retriever_dict={"vector": vector_retriever},
#     node_dict=node_mappings_charte,
#     verbose=True,
# )
# query_engine = RetrieverQueryEngine.from_args(recursive_retriever)
query_engine = RetrieverQueryEngine.from_args(vector_retriever)

In [82]:
response = query_engine.query("Quel est le code et la classification (rouge, vert, orange) du Chlorure d'étain?")
print(str(response))

Le code du Chlorure d'étain est E 512 et il est classé dans les additifs rouges.


In [83]:
response = query_engine.query("Quel est le code et la classification (rouge, vert, orange) du Bleu patenté V?")
print(str(response))

Le code du Bleu patenté V est E 131 et il est classé comme additif orange.


In [84]:
response = query_engine.query("Est ce que la Cire de carnauba est un additif de type orange? Sinon de quel type est il? Quelle est son code E ?")
print(str(response))

La Cire de carnauba n'est pas un additif de type orange. Elle est classée comme un additif vert, identifié comme non dangereux pour la santé. Son code E est E 903.


In [85]:
response = query_engine.query("Est ce que l'additif E250 est autorisé dans les produits haut de gamme d'après les exigences recette? Explique ta reflexion en précisant la couler si c'est un additif (note que la mention à éviter signifie que c'est autorisé)")
print(str(response))

L'additif E250 (nitrite de sodium) est interdit dans les produits haut de gamme, sauf dans les produits de salaison. Cela signifie que, en général, cet additif n'est pas autorisé dans les produits haut de gamme, sauf pour cette exception spécifique.


In [186]:
len(base_nodes_charte)

29